<a href="https://colab.research.google.com/github/mrslacklines/covidqa/blob/main/covidqa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center>
<img src="https://drive.google.com/uc?id=1gF_CtQygk1RDUdq1IuUsvOyNSf4nWIad" alt="covidqa" width="200"/>
</center>

# TL;DR
[Do kodu](#solution)

# NLP Developer - zadanie rekrutacyjne

Oryginalna treść zadania:

https://docs.google.com/document/d/1EgZno0N_Lggn8DTYu_kF7R0CaFr9_uoc-DhINycH_M0/edit?usp=sharing

## Problem:
Dysponujesz zbiorem dokumentów tekstowych, które chciałabyś/chciałbyś wyszukiwać za pomocą zapytań w postaci naturalnych pełnych zdań, które powiązane są z tym czego dotyczą same dokumenty. 

## Przykład:
**użytkownik**: *skąd wezmę wynik testu?*

**system:** (odpowiedź na pytanie: “Gdzie znajdę wynik testu?”) Informację o wyniku testu znajdziesz na Internetowym Koncie Pacjenta.(...)

Przykładowy zbiór pytań i odpowiedzi:
QA_covid - Rekrutacja NLP Deweloper
https://docs.google.com/spreadsheets/d/1NqcXl4m5ci9QgnrMBOoQ3Nis1hwxIpg37QblVvIGhwU/edit?usp=sharing (kopia)

## Twoje zadanie:
Zaprojektuj (wybierz algorytmy, wypisz narzędzia z których byś skorzystała/skorzystał) 3 różne rozwiązania problemu.
Zaimplementuj jedno z zaprojektowanych rozwiązań.

Nie zależy nam na największej skuteczności zaimplementowanego rozwiązania, ale na tym żeby zobaczyć w jaki sposób pracujesz, jak projektujesz oraz implementujesz rozwiązania.

Napisz jakie jeszcze operacje można byłoby wykonać, jakich narzędzi użyć żeby poprawić wyniki osiągnięte przez Twoją implementację.

Jeżeli brakuje w zadaniu jakiś założeń - sama/sam zrób założenie i skomentuj je w notebooku. Jeżeli będziesz miała/miał jakiekolwiek pytania/wątpliwości - napisz na aleksander.obuchowski@talkie.ai

Prześlij rozwiązanie w formie notebooka na Google Colab, poprosimy Cię o przedstawienie rozwiązania na naszym następnym spotkaniu.


<center>
<img src="https://drive.google.com/uc?id=19XZZAygrwhG0I1Zh_cRz85s8BBXT8R9H" alt="covidqa" width="100"/>
</center>
</br>

# Przyjęte założenia:

1. W przykładzie podanym w treści zadania odpowiedź systemu zawiera:
  1. Treść pytania o najbardziej zbliżonej treści ze zbioru pytań i odpowiedzi i
  2. pierwsze zdanie odpowiedzi.
  
  W tym przypadku założyłem, że opdowiedź została skrócona w celu poprawy czytelności a nie jako sugestia budowy systemu *Extractive* QA.

2. Opis zadania zawiera również stwierdzenie: *Nie zależy nam na największej skuteczności zaimplementowanego rozwiązania,*

  Skupiłem się raczej na rozważeniu różnych rozwiązań oraz wypisaniu możliwych usprawnień oraz ich wad i zalet niż na fine tuningu-modelu.

3. Brak wzmianek o konieczności formalnej ewaluacji modelu oraz założenie 2. przyjąłem dosłownie i potraktowałem to z najniższym priorytetem (opcjonalnie), mając pełną świadomość popełnianego błedu w sztuce.

4. Założyłem, że zależy nam na jak najszybszym rozwiązaniu problemu biznesowego i dostarczenie względnie użytecznego MVP, którego użytkownikiem końcowym będzie np. gość portalu e-Pacjent lub podobnego, który szuka odpowiedzi na typowe pytanie dotyczące wirusa. Założyłem też, że jesteśmy w trakcie trwania pandemii i pytania będą adekwatne do zbioru, który stanowi bazę dokumentów w tym zadaniu. Czyl będą to np. pytania "Czy maseczki chronią przed zakażeniem".

<center>
<img src="https://drive.google.com/uc?id=19XZZAygrwhG0I1Zh_cRz85s8BBXT8R9H" alt="covidqa" width="100"/>
</center>
</br>

# Rozwiązania

Dostępne rozwiązania można podzielić na:
- rozwiązania oparte o symboliczną reprezentację wiedzy,
- metody oparte o wyszukiwanie w przestrzeni wektorów cech:
  - oparte o klasyczne nielingwistyczne metody reprezentacji dokumentów (BOW, TFIDF),
  - oparte o reprezentację dokumentów za pomocą sieci neuronowych (embeddings), (doc2vec, transformers, sentence transformers),
- metody generatywne oparte o głębokie sieci neuronowe.

Granica pomiędzy tymi typami rozwiązań jest umowna i jest to raczej kontinuum technik, które stosowane są wybiórczo w zależności od natury problemu. Dlatego należałoby rozpatrzyć trzecią kategorię, czyli:

- Rozwiązania hybrydowe, łączące cechy pozostałych.

Rozpatrując możliwe rozwiązania pominiemy też ostatni typ, czyli w pełni generatywne modele oparte o głębokie sieci neuronowe. Są to metody, które praktycznie zawsze wymagają uczenia. W naszym przypadku dysponujemy bardzo małym zbiorem danych. Można by oczywiście pokusić się o generowanie próbek syntetycznych czy inne metody ale jest to rozwiązanie ryzykowne, które prawdopodobnie nie pozwoliło by szybko osiągnąć akceptowalnych rezultatów.

Skupimy się głównie na metodach wyszukiwawczych. W tym przypadku zwykle dysponujemy jakąś bazą wiedzy jak np. nasz zbiór FAQ, w którym wyszukiwane są odpowiedzi na pytania. Wyszukiwanie może odbywać na zasadzie próby odnalezienia (1) podobnych pytań w bazie lub (2) odpowiedzi pasujących do pytania. Mówimy wtedy o wyszukiwaniu symetrycznym lub asymetrycznym. Ja skupię się na rozwiązaniach symetrycznych, czyli będę starał się odnaleźć jak najbardziej podobne pytanie w naszej bazie. Przetestowanie działania wyszukiwania asymetrycznego jest oczywiście kuszące ale z braku czasu musimy je pominąć.

Zacznę jednak od krótkiego omówienia rozwiązania czysto lingwistycznego (w końcu nie mógłbym sobie tego podarować :)) żeby mieć to z głowy i móc przejść do rozwiązań czysto pragmatycznych.

</br>
<center>
<img src="https://drive.google.com/uc?id=19XZZAygrwhG0I1Zh_cRz85s8BBXT8R9H" alt="covidqa" width="100"/>
</center>
</br>

## Rozwiązanie oparte o symboliczną reprezentację wiedzy (1)

W niektórych przypadkach opłaca się zbudować specjalną symboliczną reprezentację wiedzy zawartej w dostępnych dokumentach. W tym celu wykorzystuje się szereg metod z zakresu *klasycznego*, które służą do ekstrakcji wiedzy i stanowią też inwentarz narzędzi Information Retrievalowych, np. wykrywanie bytów nazwanych i parsing zależnościowy.

Informacje te wykorzystywane są do ekstrakcji wiedzy i jej zapisu np. za pomocą jakiegoś języka logiki; rachunku predykatów, logiki modalnej i temporalnej. Mogą to też być np. trójki (*triplets*) Object Attribute Value, sieci semantyczne podobne do Wordnetu tylko oparte o inne interesujące nas relacje, czy też specjalnie zaprojektowane struktury danych/formalizmy takie jak ramy semantyczne (np. CxG) czy ontologie.

Ważnym elementem tak wdrożonego rozwiązania jest też przetwarzanie samych pytań na podstawie różnego rodzaju taxonomii. Także tutaj dostępne są różne rozwiązania i formalizmy, jednak cel we wszystkich przypadkach jest taki sam -- wykrycie i formalna reprezentacja typu pytania i rodzaju odpowiedzi (np. "Czy Sokrates jest ateńczykiem?" `-> ENTITY|PERSON|BIRTHPLACE`).

Po otrzymaniu i przetworzeniu pytania system stara się wyłonić kandydatów wśród dokumentów/rekordów w bazie wiedzy.

### Zalety i wady
Jak widać już na pierwszy rzut oka to rozwiązanie wymaga bardzo dużych nakładów czasu oraz mozolnej pracy, mimo tego, że możliwa jest oczywiście częściowa automatyzacja budowy bazy wiedzy oraz zaprzężenie technik neuralnych do całego procesu.
Symboliczna oraz regułowa natura reprezentacji wiedzy oraz sposobu jej przetwarzania sprawia, że gotowy system jest w pełni *wyjaśnialny* (*explainable*) ale jest on też niezbyt elastyczny i posiada bardzo ograniczone możliwości generalizacji.

### Technologie

Ilość narzędzi i zasobów do symbolicznego przetwarzania języka polskiego jest wbrew pozorom całkiem spora. Zespół badaczy z ZIL IPI PANu oraz projektu CLARIN a także wielu pokrewnych projektów i grup badawczych udostępnia je na otwartych licencjach (co może być zarówno ich wadą jak i zaletą).

http://clarin-pl.eu

http://zil.ipipan.waw.pl

Warto tutaj wymienić:
- Polski Wordnet
- Morfeusz
- Concraft
- Polish Dependency Parsing i polski Treebank
- NLTK, SpaCy

Myślę, że gdyby było więcej czasu można by pokusić się o zbudowanie prototypu w oparciu o te narzędzia, do reprezentacji wiedzy wykorzystując triplety OAV i dowolny framework do rozwoju gramatyk, np. Grammatical Framework.
Wartość takiego rozwiązania dla bieżącego problemu oceniam jednak dość nisko. Było by ono na pewno ciekawe z badawczego punktu widzenia :)


<center>
<img src="https://drive.google.com/uc?id=19XZZAygrwhG0I1Zh_cRz85s8BBXT8R9H" alt="covidqa" width="100"/>
</center>


## Rozwiązania oparte o wyszukiwanie w przestrzeni wektorów cech

## Wyszukiwanie oparte o TF-IDF (2)

Kolejna klasa możliwych rozwiązań opiera się o algorytmy używane w klasycznym przetwarzaniu tekstu. Stosują one popularne od wielu lat metody parametryzacji dokumentów tekstowych za pomocą zliczania częstotliwości występujących w nich słów. Stosuje się tu szereg metod, również core'owych metod NLP. W przypadku syntetycznego (fleksyjnego) języka polskiego opłacalne jest bowiem sprowadzenie słów do form podstawowych. Można to zrobić metodą *płytką* poprzez stemming lub *głęboką* za pomocą lemmatyzacji. W ten sposób nie będziemy zliczać wszystkich występujących w dokumentach form osobno, zaoszczędzimy pamięć, zmniejszymy rozmiar finalnego wektora i na pewno poprawimy wyniki. Dodatkowo, okazuje się, że warto odfiltrować tzw. *stopwords*. Są to słowa bardzo częste niosące bardzo małe znaczenie jak np. zaimki osobowe, przyimki itd. (ilokroć to piszę robi mi się słabo, jako językoznawcy, bo np. zaimek osobowy "ja" jest przedmiotem dyskusji filozofów i myślicieli od tysięcy lat, no ale nie o tym.. :)).
Na tej podstawie możemy najzwyczajniej zliczyć wystąpienia wszystkich słów z naszego *vocabulary* w każdym dokumencie (bag-of-words) i przedstawić to jako wektor rzadki. Usprawnienie może być użycie n-gramów, które umożliwią nam reprezentację lokalnego (bezpośredniego) konktekstu i stanowią bardzo proste ale potężne ulepszenie. Tutaj oczywiście pojawia się problem związany z małą ilościa przykładów i wielkim rozmiarem wektora cech.
Kolejną modyfikacją tej metody jest zastosowanie parametryzacji poprzez wyliczenie TF-IDF (Text Frequency - Inverse Document Frequency). W tym przypadku ważymy słowa występujące w danym dokumencie przez pryzmat ich ogólnej częstości we wszystkich dokumentach. W ten prosty sposób możemy otrzymać reprezentację wektorową, która reprezentuje słowa charakterystyczne dla danego dokumentu w kontekście frekwencji w całym naszym zbiorze. Tutaj też możemy wykorzystać nie tylko pojedyncze słowa ale i n-gramy.
Tak przygotowaną reprezentację wektorową można wykorzystać do bardzo efektywnego wyszukiwania za pomocą miar odległości euklidesowej, podobieństwa cosinusowego czy np. miary Jaccarda. Przechowując takie reprezentacje tekstu w bazie wektorowej, która dodatkowo umożliwia wykonywanie kwerend w oparciu o te miary jest bardzo efektywne i może zostać bardzo szybko zdeployowane w postaci bardzo taniego i wysokoskalowalnego rozwiązania chmurowego (np. serverless)
Takie właśnie klasyczne rozwiązanie naszego problemu (bez cloud deploymentu :)) zaimplementujemy jako benchmark, z którym porównamy metody oparte o reprezentacje wektorowe generowane przez najlepsze sieci neuronowe.

### Zalety i wady
Rozwiązanie to jest bardzo proste i szybkie do zaimplementowania. To w zasadzie dwie linijki w SKLearn. Jest ono bardzo łatwo *wyjaśnialne* oraz bardzo elastyczne. Posiada bardzo dużą zdolność generalizowania. Co prawda wyliczanie TF-IDF jest dość wolne ale przy infrastrukturze i zasobach, której wymaga współczesny deep learning, myślę, że to już trochę wstyd o tym wspominać.
Najwiekszą wadą tego rozwiązania jest ograniczona możliwość reprezentacji znaczenia. Jest ona minimalna bo zawęża się tylko do bezpośredniego kontekstu (i to w najlepszym wypadku).

### Technologie

- SKLearn
- SpaCy
- NLTK

## Wyszukiwanie semantyczne oparte o Sentence Embeddings (3)

Na drugim biegunie znajdują się rozwiązania oparte o głebokie sieci neuronowe. Stosuje się tutaj w zasadzie wszystkie architektury takie jak rekurencyjne sieci neuronowe (RNN), które do pewnego momentu stanowiły standard *de facto* dla problemów związanych z przetwarzaniem języka, sieci LSTM czy sieci CNN.
Występują one w różnorodnych konfiguracjach np. jako sieci syjamskie.

Jeżeli chodzi o klasyczny *representation learning* trzeba oczywiście koniecznie wspomnieć o Gensim Word2Vec i wszystkich jego następcach i odmianach jak Doc2Vec, Sent2Vec czy np. Graph2Vec. Oryginalny Gensim to w zasadzie dwa modele, stanowiące niejako swoje lustrzane odbicie; Skip-Gram oraz Continuous Bag Of Words.

W 2023 roku trudno jednak pominąć sieci typu Transformer i opartych o te architektury Pretrained Large Language Models (PLLMs). Podobnie jak Doc2Vec pozwalają one wygenerować parametryczną reprezantację dokumentu tekstowego w postaci *gęstego* wektora. Charakter tych modeli sprawia, że taki w zasadzie dość nieduży (np. 768) wektor w zaskakujący sposób oddaje charakter języka i kontekstualne (distrybucyjne) znaczenie słów lub innych tokenów.

Taka reprezentacja dokumentów umożliwia ich wyszukiwanie i porównywanie za pomocą tych samych miar jak w przypadku rozwiązania opartego o TF-IDF.

**Poniżej, jako rozwiązanie zadania, znajduje się prosta implementacja metody opartej o Sentence-Transformers, indeksowanie FAISS i badania podobieństwa cosinusowego**
[Do kodu](#solution)

### Zalety i wady
Jest to chyba najdynamiczniej rozwijająca się dziedzina Machine Learningu. Powstało i wciąż powstaje wiele wspaniałych bibliotek oraz publicznie dostępnych modeli. Jednym z najważniejszych ekosystemów jest HuggingFace. Oczywiście język polski jest jak zwykle lekko z tyłu i chociaż istnieją wytrenowane modele oparte o te najpopularniejsze architektury to daleko im do tych dla języka angielskiego czy chińskiego. Modele te są zwykle bardzo duże, koszt ich treningu jest wielki. Jesteśmy więc zdani na to co jest dostępne.
Transfer learning otwiera przed nami zupełnie nowe możliwości. Możemy wykorzystać core polskiego HerBerta wytrenowanego na MLM i dodać mu *HEAD* np. do QuestionAnsweringu, który będzie wymagał tylko małego dotrenowania i uzyskamy potężne rozwiązanie stosunkowo małym kosztem.
Zarówno wad i zalet jest tutaj bardzo dużo i trudno wyczerpać temat w ramach kilku zdań.

### Technologie

- HuggingFace

## Usprawnienia (+ 4 rozwiązanie :))

Ponieważ i tak już jest zbyt dużo tekstu to wymienie potencjalne usprawnienia w formie punktów. Ostatnie z nich można potraktować jako 4 dodatkowy sposób rozwiązania problemu.

- Rozwijanie zapytań poprzez parafrazę
- Dodanie modułu rule-based dla najbardziej typowych pytań, np. w oparciu o słowa kluczowe albo nawet RegExpy
- Post-processing zwracanych odpowiedzi
- Wyłanianie n-kandydatów i dodatkowy algorytm do ratingu kandydatów
- Rozwój metody do ewaluacji algorytmu (uważam, że jego brak jest największym pain pointem)
- Fajnie było by zrobić mini grid search przez różne modele i miary podobieństwa/odległości jak:
  - Podobieństwo cosinusowe,
  - Euclidean distance
  - Jaccard Index
- Można by pokusić się o próbę transfer learningu jakiegoś z dostępnych modeli. Może mógłby to być model domenowy (BioMed) albo model typowo QuestionAnsweringowy?
- Widziałem też rozwiązanie dla francuskiego, w którym wykorzystano model dla języka angielskiego w połączeniu z modelem do tłumaczenia maszynowego.
- Jest trochę angielskich datasetów dla Covid19 i/lub dla Biomed QA. Może udałoby się uzyskać ciekawe rezultaty tłumacząc taki dataset automatycznie na polski i wyuczając na nim jakiś model.
- Są też różne datasety QA dla polskiego a nawet framework testowy KLEJ, którym nie poświęciłem należytej uwagi.
- **Rozwiązanie 4**: Zbudowanie kilku modeli w oparciu o różne metody oraz modele (np. TFIDF, Doc2Vec, BERT, SentenceBert itp.) i zestackowanie ich w ramach jednego algorytmu. Na końcu można by wykorzystać nawet tak proste rozwiązanie jak głosowanie większościowe. Kilka słabych modeli razem jest lepsze niż najlepszy z nich osobno. Można by pokusić się też o Metalearnera na końcu tego stacku, sieci syjamskie, tyle tego jest.. :)

<a name="solution"></a>
# Rozwiązanie oparte o Semantic Search na podstawie Sentence Embeddings

To rozwiązanie wydało mi się najbardziej sensowne ponieważ:

1. Pozwala szybko uzyskać względnie dobre rezultaty
2. Było łatwe do zaimplementowania dzięki wykorzystaniu HuggingFace
3. Dostępne modele dla polskiego są wystarczające dla tego rozwiązania, jest ich kilka i można było je łatwo przetestować
4. Treść zadania nie narzuca budowy systemu typu Generative QA czy nawet Extractive QA.

Korzystam głównie z HuggingFace Transformers. Instaluję biblioteki, którymi się posługiwałem.

In [1]:
!pip install transformers accelerate sentence_transformers datasets faiss-gpu
!python -m spacy download pl_core_news_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2023-02-20 19:24:18.190765: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-20 19:24:20.077762: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-20 19:24:20.077914: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nv

Importuję zainstalowane biblioteki.

In [2]:
from google.colab import auth
import pandas as pd

import torch
from sentence_transformers import SentenceTransformer, util
from datasets import Dataset
from transformers import AutoTokenizer, AutoModel


Tutaj definiuję kilka *globalny stałych* (zmiennych), tak żeby nie hardcodować np. ścieżki do pliku z danymi czy nazwy checkpointa HF.

In [3]:
DATA_FILENAME = "QA_covid - Rekrutacja NLP Deweloper"
SHEET_NAME = "q_a (1)"
MODEL_CHECKPOINT = "Voicelab/sbert-large-cased-pl"

device = torch.device("cuda")

Loguje się do Google Drive gdzie zapisałem arkusz z pytaniami i odpowiedziami.

In [4]:
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

Dzieki temu mogę w kilku linijkach kodu odczytać arkusz prosto z chmury i wczytać go jako DataFrame Pandas.

In [5]:
worksheet = gc.open(DATA_FILENAME).sheet1

df = pd.DataFrame.from_records(worksheet.get_all_values())
df.columns = df.iloc[0]
df.drop(index=0, inplace=True)
df_backup = df.copy()
df

,Q,A
1,Jakie objawy są charakterystyczne dla koronawi...,Osoby zakażone koronawirusem zwykle mają:wysok...
2,Co zrobić w przypadku wystąpienia duszności i ...,Jeżeli występują u ciebie duszności lub kłopot...
3,Mam objawy charakterystyczne dla COVID-19. Co ...,Jeśli Twoje objawy są typowe dla zakażenia kor...
4,Nie mogę skontaktować się ze swoim lekarzem PO...,Jeśli nie możesz skontaktować się ze swoim lek...
5,"Mój lekarz POZ nie dyżuruje, a ja mam objawy C...",Jeśli lekarz wybrany przez Ciebie jako lekarz ...
...,...,...
331,Czy regularne płukanie nosa solą fizjologiczną...,"Nie. Nie ma dowodów na to, że regularne płukan..."
332,Czy jedzenie czosnku może zapobiec infekcji no...,"Czosnek to zdrowa żywność, która może mieć pew..."
333,Czy zimna pogoda i śnieg mogą zabić koronawirusa?,"Nie ma powodu sądzić, że zimna pogoda może zab..."
334,Czy osoba która umiera na COVID podlega kremacji?,Nie ma obowiązku kremacji zmarłych na COVID-19.


Jeżeli chcielibyśmy np. fine tune'ować model łatwo da się też przekonwertować dane do postaci Datasetu HuggingFace, który też udostępnia ograniczone API do transformacji danych (np. metodę `map()`) 

In [6]:
ds = Dataset.from_pandas(df)
ds

Dataset({
    features: ['Q', 'A', '__index_level_0__'],
    num_rows: 335
})

Tutaj wykorzystuję bardzo przydatne API HuggingFace w klasach `AutoModel` i `AutoTokenizer`, które bardzo przyśpieszają prototypowanie i eksperymenty z różnymi checkpointami i architekturami modeli. HF jest w stanie automatycznie wykryć architekturę modelu i go zainicjować. Dostępne są też klasy typu `AutoModelFor*` np. `AutoModelForQuestionAnswering`, które wykorzystują core dowolnego modelu i dodają do niego automatycznie wygenerowany *Head* dla konkretnego zadania. W naszym przypadku nie jest to koniecznie bo będzie wykorzystywali tylko enkoder do budowy reprezentacji semantycznej tekstu. Wykorzystany tutaj model to SentenceBert. Pod spodem HF wykorzystuje specjalną libkę Sentence-Bert https://www.sbert.net/docs/usage/semantic_textual_similarity.html. W pierwszej iteracji tego rozwiązania też z niej korzystałem ale uznałem, że nie ma sensu utrudniać sobie życia :) Swoją drogą w trakcie przygotowywania się do realizacji tego zadania zrobiłem kurs HuggingFace Transformers i stałem się wielkim fanem :)
W ostatnim kroku ładujemy model do GPU bo szanujemy swój czas.

In [7]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)
model = AutoModel.from_pretrained(MODEL_CHECKPOINT)
model.to(device)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(50000, 1024, padding_idx=1)
    (position_embeddings): Embedding(514, 1024)
    (token_type_embeddings): Embedding(2, 1024)
    (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=1024, out_features=1024, bias=True)
            (key): Linear(in_features=1024, out_features=1024, bias=True)
            (value): Linear(in_features=1024, out_features=1024, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=1024, out_features=1024, bias=True)
            (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=Fals

Definiujemy sobie funkcję pomocniczą do *poolowania* i do parametryzacji tekstu pod postacią embeddingsów.

In [8]:
def cls_pooling(model_output):
    return model_output.last_hidden_state[:, 0]

In [9]:
def get_embeddings(text_list):
    encoded_input = tokenizer(
        text_list, padding=True, truncation=True, return_tensors="pt"
    )
    encoded_input = {k: v.to(device) for k, v in encoded_input.items()}
    model_output = model(**encoded_input)
    return cls_pooling(model_output)

Generujemy sobie dataset pod postacią samych embeddingsów i dodajemy index FAISS (korzystałem kiedyś z niego w projekcie z rozpoznawaniem twarzy). W przypadku budowy prawdziwego rozwiązania można w ramach optymalizacji należało by to było wrzucić do bazy danych, która:
- wspiera taki format danych (wektory)
- implementuje podobieństwo cosinusowe lub inną podobną miarę jako operację za pomocą, której możemy wyszukiwać rekordy.

In [10]:
embeddings_dataset = ds.map(
    lambda x: {"embeddings": get_embeddings(x["Q"]).detach().cpu().numpy()[0]}
)

  0%|          | 0/335 [00:00<?, ?ex/s]

In [11]:
embeddings_dataset.add_faiss_index(column="embeddings")

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset({
    features: ['Q', 'A', '__index_level_0__', 'embeddings'],
    num_rows: 335
})

Możemy już sprawdzić jak działa wyjściowy system.

In [12]:
#@title Wpisz pytanie
question = 'Czy wietrzenie pomieszczenia zapobiega rozprzestrzenianiu si\u0119 koronawirusa?' #@param {type:"string"}


In [13]:
print(question)

Czy wietrzenie pomieszczenia zapobiega rozprzestrzenianiu się koronawirusa?


Jak widać system dość dobrze radzi sobie ze znalezieniem podobnych pytań w bazie na poziomie słownictwa - niestety nie zawsze oznacza to znalezienie najbardziej odpowiedniej odpowiedzi na zadane pytanie.

In [68]:
def print_answer(question, matched_db_question, answer):
  print(f"INPUT QUESTION: {question}")
  print(f"MATCHED DB QUESTION: {matched_db_question}")
  print(f"ANSWER: {answer}")

In [90]:
question_embedding = get_embeddings([question]).cpu().detach().numpy()

scores, samples = embeddings_dataset.get_nearest_examples(
    "embeddings", question_embedding, k=5
)
samples_df = pd.DataFrame.from_dict(samples)
samples_df["scores"] = scores
samples_df.sort_values("scores", ascending=False, inplace=True)

best_ans = samples_df.iloc[0]

print_answer(question, best_ans.Q, best_ans.A)

INPUT QUESTION: Czy wietrzenie pomieszczenia zapobiega rozprzestrzenianiu się koronawirusa?
MATCHED DB QUESTION: Czy lampa dezynfekująca na promienie UV może zabić nowego koronawirusa?
ANSWER: Lampy UV przeznaczone są do dezynfekcji pomieszczeń, nie powinny być używane do sterylizacji rąk lub innych obszarów skóry, ponieważ promieniowanie UV może powodować podrażnienie skóry.


Dla zabawy możemy też zobaczyć jak poradziłby sobie model do ekstraktywnego QA:

In [15]:
from transformers import pipeline

extractive_qa_pipeline = pipeline(
    "question-answering",
    model="henryk/bert-base-multilingual-cased-finetuned-polish-squad2",
    tokenizer="henryk/bert-base-multilingual-cased-finetuned-polish-squad2"
)

In [16]:
context = best_ans.A

In [74]:
extr_ans = extractive_qa_pipeline(
    {
        'context': context,
        'question': question
     }
)

print_answer(question, context, extr_ans.get('answer', '???'))



INPUT QUESTION: Czy wietrzenie pomieszczenia zapobiega rozprzestrzenianiu się koronawirusa?
MATCHED DB QUESTION: Nie ma powodu sądzić, że zimna pogoda może zabić koronawirusa lub inne choroby. Normalna temperatura ciała człowieka wynosi około 36,5 stopni C do 37 stopni C, niezależnie od temperatury zewnętrznej i pogody. Najskuteczniejszym sposobem ochrony przed nowym koronawirusem jest częste mycie rąk wodą i mydłem oraz korzystanie środków do dezynfekcji rąk na bazie alkoholu . Należy także unikać bliskiego kontaktu z każdym, kto kaszle i kicha.
ANSWER: zimna pogoda może zabić koronawirusa lub inne choroby.


Nie dysponujemy niestety żadnym zbiorem testowym, na którym można by przetestować działanie naszego rozwiązania. Pozostaje nam ręczne testowanie i trochę pośrednich miar, które można by wyliczyć.

Zapomnieliśmy jednak o najważniejszym kroku, od którego powinniśmy zacząć. Za bardzo poniosły nas emocje i nie zbudowaliśmy prostego modelu, który mógłby stanowić nasz benchmark. Zrobimy to teraz.

W tym celu użyję miary TF-IDF, na podstawie której zbudujemy wektor cech składający się z frekwencji słów kluczowych każdego dokumentu. Następnie wykorzystamy te wektory w ten sam sposób jak poprzednio - będziemy wyszukiwali podobne pytania za pomocą np. podobieństwa cosinusowego.

In [75]:
import spacy
from spacy.lang.pl.examples import sentences 

nlp = spacy.load("pl_core_news_sm")

# Usuwamy niepotrzebne elementy pipeline'u
print("Pipeline NLP SpaCy")
print("Przed: ", ", ".join(nlp.pipe_names))
for proc_step in ["parser", "attribute_ruler", "ner"]:
  nlp.remove_pipe(proc_step)
print("Po: ", ", ".join(nlp.pipe_names))

Pipeline NLP SpaCy
Przed:  tok2vec, morphologizer, parser, lemmatizer, tagger, attribute_ruler, ner
Po:  tok2vec, morphologizer, lemmatizer, tagger


In [76]:
def preprocess(doc):
  return [tok.lemma_ for tok in nlp(doc) if not tok.is_stop and not tok.is_punct]

In [78]:
df["Q_preprocessed"] = df["Q"].apply(preprocess)
df["Q_preprocessed"].values[:20]

array([list(['objaw', 'charakterystyczny', 'koronawiruso']),
       list(['zrobić', 'przypadek', 'wystąpić', 'duszność', 'trudność', 'oddychanie']),
       list(['objaw', 'charakterystyczny', 'COVID', '19', 'powinien być', 'zrobić', 'pierwszy', 'kolejność']),
       list(['móc', 'skontaktować', 'swój', 'lekarz', 'POZ', 'zrobić']),
       list(['lekarz', 'POZ', 'dyżurować', 'objawy', 'COVID', '19', 'zrobić']),
       list(['skierować', 'test', 'wystawić', 'lekarz', 'POZ', 'lekarz', 'nocny', 'świąteczny', 'telepomoc', 'wystawić', 'skierowanie']),
       list(['znaleźć', 'wynik', 'test']),
       list(['wynik', 'test', 'pozytywny', 'zostana', 'poinformować']),
       list(['skontaktować być', 'sanepidem', 'pomoc', 'formularz', 'strona', 'dostać', 'odpowiedź', 'pytanie']),
       list(['znaczyć', 'mieć być', 'bliski', 'kontakt', 'osoba', 'chory']),
       list(['mieć być', 'bliski', 'kontakt', 'osoba', 'otrzymać', 'pozytywny', 'wynik', 'test', 'powinien być', 'zgłosić']),
       list(['dos

In [79]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Pomijamy wbudowany preprocesor i użyjemy uni- i bigramów żeby oszukać i przemycić trochę semantyki dystrybucyjnej ;)
tfidf = TfidfVectorizer(ngram_range=(1, 2), tokenizer=preprocess, stop_words=None, lowercase=False)    
tfidf_docs_arr = tfidf.fit_transform(df["Q"])

Możemy sprawdzić czy nasz `TfidfVectorizer` zadziałał jak trzeba sprawdzając vocabulary.

In [88]:
list(tfidf.vocabulary_)[:100]

['objaw',
 'charakterystyczny',
 'koronawiruso',
 'objaw charakterystyczny',
 'charakterystyczny koronawiruso',
 'zrobić',
 'przypadek',
 'wystąpić',
 'duszność',
 'trudność',
 'oddychanie',
 'zrobić przypadek',
 'przypadek wystąpić',
 'wystąpić duszność',
 'duszność trudność',
 'trudność oddychanie',
 'COVID',
 '19',
 'powinien być',
 'pierwszy',
 'kolejność',
 'charakterystyczny COVID',
 'COVID 19',
 '19 powinien być',
 'powinien być zrobić',
 'zrobić pierwszy',
 'pierwszy kolejność',
 'móc',
 'skontaktować',
 'swój',
 'lekarz',
 'POZ',
 'móc skontaktować',
 'skontaktować swój',
 'swój lekarz',
 'lekarz POZ',
 'POZ zrobić',
 'dyżurować',
 'objawy',
 'POZ dyżurować',
 'dyżurować objawy',
 'objawy COVID',
 '19 zrobić',
 'skierować',
 'test',
 'wystawić',
 'nocny',
 'świąteczny',
 'telepomoc',
 'skierowanie',
 'skierować test',
 'test wystawić',
 'wystawić lekarz',
 'POZ lekarz',
 'lekarz nocny',
 'nocny świąteczny',
 'świąteczny telepomoc',
 'telepomoc wystawić',
 'wystawić skierowanie

In [85]:
def get_tfidf_sim(vectorizer, docs_tfidf, query):
    query_tfidf = vectorizer.transform([query])
    cosineSimilarities = cosine_similarity(query_tfidf, docs_tfidf).flatten()
    return cosineSimilarities

In [89]:
top_sim_index = get_tfidf_sim(tfidf, tfidf_docs_arr, question).argmax()

matched_question = df["Q"].iloc[min_distance_vector_index]
matched_answer = df["A"].iloc[min_distance_vector_index]
print_answer(question, matched_question, matched_answer)

INPUT QUESTION: Czy wietrzenie pomieszczenia zapobiega rozprzestrzenianiu się koronawirusa?
MATCHED DB QUESTION: Jak zdezynfekować pomieszczenia w zakładzie pracy, w którym przebywała osoba chora lub podejrzana o zachorowanie na koronawirusa? Kto powinien potwierdzić, że ta dezynfekcja miała miejsce?
ANSWER: Osoba odpowiedzialna za dezynfekcję pomieszczenia oraz sposób jej dokumentacji powinna być określona przez pracodawcę. W sprawie dezynfekcji pomieszczenia możemy posiłkować się wytycznymi GIS dla POZ (dotyczące dezynfekcji pomieszczenia, w której przebywała osoba chora lub podejrzana o zachorowanie na koronawirusa), tj. wyłączyć z funkcjonowania pomieszczenie, w którym przebywała osoba zakażona, umyć i zdezynfekować powierzchnie, meble, sprzęt – po dezynfekcji może być ponownie używane, wywietrzyć pomieszczenie, w którym przebywała osoba zakażona, zdezynfekować drogę lub drogi dojścia do pomieszczenia gdzie przebywała osoba zakażona do pomieszczenia (poręcze schodów, klamki, lady i

Wygląda na to, że musimy dodać wszystkie określenia koronawirusa do stopwordsów bo algorytm jest na nie zbyt wrażliwy

In [91]:
domain_specific_stopwords = [
    "koronawiruso",
    "COVID",
    "19"
]

In [93]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Pomijamy wbudowany preprocesor i użyjemy uni- i bigramów żeby oszukać i przemycić trochę semantyki dystrybucyjnej ;)
tfidf = TfidfVectorizer(ngram_range=(1, 2), tokenizer=preprocess, stop_words=domain_specific_stopwords, lowercase=False)    
tfidf_docs_arr = tfidf.fit_transform(df["Q"])

In [94]:
list(tfidf.vocabulary_)[:100]

['objaw',
 'charakterystyczny',
 'objaw charakterystyczny',
 'zrobić',
 'przypadek',
 'wystąpić',
 'duszność',
 'trudność',
 'oddychanie',
 'zrobić przypadek',
 'przypadek wystąpić',
 'wystąpić duszność',
 'duszność trudność',
 'trudność oddychanie',
 'powinien być',
 'pierwszy',
 'kolejność',
 'charakterystyczny powinien być',
 'powinien być zrobić',
 'zrobić pierwszy',
 'pierwszy kolejność',
 'móc',
 'skontaktować',
 'swój',
 'lekarz',
 'POZ',
 'móc skontaktować',
 'skontaktować swój',
 'swój lekarz',
 'lekarz POZ',
 'POZ zrobić',
 'dyżurować',
 'objawy',
 'POZ dyżurować',
 'dyżurować objawy',
 'objawy zrobić',
 'skierować',
 'test',
 'wystawić',
 'nocny',
 'świąteczny',
 'telepomoc',
 'skierowanie',
 'skierować test',
 'test wystawić',
 'wystawić lekarz',
 'POZ lekarz',
 'lekarz nocny',
 'nocny świąteczny',
 'świąteczny telepomoc',
 'telepomoc wystawić',
 'wystawić skierowanie',
 'znaleźć',
 'wynik',
 'znaleźć wynik',
 'wynik test',
 'pozytywny',
 'zostana',
 'poinformować',
 'test 

In [ ]:
list(tfidf.vocabulary_)[:100]

In [95]:
top_sim_index = get_tfidf_sim(tfidf, tfidf_docs_arr, question).argmax()

matched_question = df["Q"].iloc[min_distance_vector_index]
matched_answer = df["A"].iloc[min_distance_vector_index]
print_answer(question, matched_question, matched_answer)

INPUT QUESTION: Czy wietrzenie pomieszczenia zapobiega rozprzestrzenianiu się koronawirusa?
MATCHED DB QUESTION: Jak zdezynfekować pomieszczenia w zakładzie pracy, w którym przebywała osoba chora lub podejrzana o zachorowanie na koronawirusa? Kto powinien potwierdzić, że ta dezynfekcja miała miejsce?
ANSWER: Osoba odpowiedzialna za dezynfekcję pomieszczenia oraz sposób jej dokumentacji powinna być określona przez pracodawcę. W sprawie dezynfekcji pomieszczenia możemy posiłkować się wytycznymi GIS dla POZ (dotyczące dezynfekcji pomieszczenia, w której przebywała osoba chora lub podejrzana o zachorowanie na koronawirusa), tj. wyłączyć z funkcjonowania pomieszczenie, w którym przebywała osoba zakażona, umyć i zdezynfekować powierzchnie, meble, sprzęt – po dezynfekcji może być ponownie używane, wywietrzyć pomieszczenie, w którym przebywała osoba zakażona, zdezynfekować drogę lub drogi dojścia do pomieszczenia gdzie przebywała osoba zakażona do pomieszczenia (poręcze schodów, klamki, lady i

Powinno być troszkę lepiej. :)

Możemy teraz porównać sobie nasze dwa rozwiązania uznając, że to oparte o TF-IDF jest benchmarkiem. brakuje nam zbioru danych i miary ale możemy potestować ręcznie.

Skoro obydwa rozwiązania mają swoje wady i zalety, które dodatkowo różnią się między sobą, czemu nie pozwolić im współpracować korzystając faktu, że np. zespół słabych klasyfikatorów osiąga dużo lepsze wyniki niż każdy z osobna.

Sprawdźmy jak będzie działało rozwiązanie, które łączyło będzie wyniki kilku rozwiązań poprzez proste głosowanie większościowe (jak np. w modelach random forest).

